<div style="text-align: center;">
  <img src="../../../assets/Cover.png" alt="WTF-Agents cover">
</div>

上一节：**[HelloWorld 构建你的第一个智能体](../A01-HelloWorld/HelloWorld.ipynb)**


### 章节介绍

本节内容我们学习一个特别重要的概念`Message`. `Message` 是 AutoGen 中的一个核心概念，它定义了智能体之间以及智能体内部信息传递的标准和格式，同时我们也可以通过继承 `BaseChatMessage` 和 `BaseAgentEvent`来自定义自身应用场景的消息格式和行为。在编写自定义代理时，自定义消息类型非常有用！

### 1. Message 模块介绍

在AutoGen中 `Message` 模块是实现与其他智能体、协调器及应用程序之间通信和信息交换的媒介,从总体上看 `AgentChat` 中的消息可分为两大类：智能体间消息和智能体内部事件消息。

<div style="text-align: center;">
  <img src="./assets/messages.png" alt="autogen messages structure", style="width: 60%;">
</div>

### 2. Agent-Agent Messages  智能体间消息

AgentChat 支持多种智能体间通信的消息类型，这些类型均继承自基类 `BaseChatMessage` 。`BaseChatMessage` 是用于智能体间通信的抽象基类，定义了消息的基本结构和行为。它的子类用于处理不同类型的消息内容例如：

+ `TextMessage`- 纯文本消息；
+ `MultiModalMessage` - 多模态消息；
+ `StopMessage` - 停止消息；
+ `ToolCallSummaryMessage`  - 工具调用摘要消息；
+ `HandoffMessage` - 任务传递消息；
+ `StructuredMessage`  - 泛型结构化消息。





2.1. 以下代码片段展示了如何创建`TextMessage`文本消息，它接受字符串内容和字符串来源，用于传输简单的文本消息。

In [1]:
from autogen_agentchat.messages import TextMessage

# 创建一个 TextMessage 实例
text_message = TextMessage(content="Hello, world!", source="User")

# 打印消息内容和来源
print(f"Content: {text_message.content}")
print(f"Source: {text_message.source}")

Content: Hello, world!
Source: User


**解释**：
- `content` 参数是一个字符串，表示消息的主要内容。
- `source` 参数标识消息的发送者（例如 "User" 或某个智能体的名称）。
- `TextMessage` 适用于简单的文本交互场景，例如用户向智能体发送指令或智能体之间的文本对话。


2.2. `MultiModalMessage` 支持多模态内容，允许消息包含文本和图像（或其他多模态数据）的组合。内容以列表形式传递，列表元素可以是字符串或 `Image` 对象。

In [2]:
from io import BytesIO
import requests
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image

# 从 URL 获取图像
pil_image = Image.open(BytesIO(requests.get("https://picsum.photos/300/200").content))
img = AGImage(pil_image)

# 创建一个 MultiModalMessage 实例
multi_modal_message = MultiModalMessage(
    content=["Can you describe the content of this image?", img],
    source="User"
)

# 打印消息内容和来源
print(f"Content: {multi_modal_message.content}")
print(f"Source: {multi_modal_message.source}")

Content: ['Can you describe the content of this image?', <autogen_core._image.Image object at 0x000001B342489030>]
Source: User


**解释**：
- `content` 是一个列表，可以包含字符串和 `AGImage` 对象（AutoGen 的图像类）。
- `source` 标识消息的发送者。
- 该消息类型适用于需要结合文本和视觉信息的场景，例如要求智能体分析图像内容。

2.3.  `StopMessage` 用于指示对话的终止，通常由智能体或用户发送以结束交互。

In [3]:
from autogen_agentchat.messages import StopMessage

# 创建一个 StopMessage 实例
stop_message = StopMessage(content="TERMINATE", source="User")

# 打印消息内容和来源
print(f"Content: {stop_message.content}")
print(f"Source: {stop_message.source}")

Content: TERMINATE
Source: User


**解释**：
- `StopMessage` 通常包含一个表示终止的字符串（例如 "TERMINATE"）。
- 智能体接收到此类消息后会停止对话或任务。
- 适用于需要明确结束多智能体交互的场景，例如用户完成任务或智能体检测到无进一步处理需求。

2.4. `ToolCallSummaryMessage` 用于总结工具调用的结果，例如智能体调用外部 API 或执行代码后的输出。

In [4]:
import json
from autogen_agentchat.messages import ToolCallSummaryMessage

# 创建一个 ToolCallSummaryMessage 实例 - JSON 字符串版本
tool_data = {"tool": "calculate_sum", "result": 42}
tool_call_summary = ToolCallSummaryMessage(
    # ToolCallSummaryMessage 的 content 字段在当前版本的 AutoGen 中期望接收字符串类型
    content=json.dumps(tool_data),  # 将字典转换为 JSON 字符串
    source="CalculatorAgent"
)

# 打印消息内容和来源
print(f"Content: {tool_call_summary.content}")
print(f"Source: {tool_call_summary.source}")


Content: {"tool": "calculate_sum", "result": 42}
Source: CalculatorAgent


**解释**：
- `content` 通常是一个字符串，包含工具调用的相关信息（如工具名称和执行结果）。
- `source` 标识调用工具的智能体。
- 这种消息类型用于智能体间共享工具执行的结果，例如在多智能体协作中，一个智能体完成计算后将结果传递给另一个智能体。

2.5. `HandoffMessage`  是 AutoGen 中用于智能体之间传递任务或上下文的特殊消息类型。它的核心作用是实现任务的移交（handoff），让一个智能体能够将当前任务转移给另一个更适合处理该任务的智能体。

`HandoffMessage`是一个使用上限很高的消息类型，它可以用于多种场景，例如：
- 在多智能体系统中，智能体A可以在任务执行失败的情况下将任务从自身中转移给智能体B；
- 可以指定消息的目标智能体，允许灵活的任务分配和协作；
- 也可以利用这种特性实现 `TaskFactory`，一个指挥官智能体批量产生新的任务，之后将任务分配给不同的执行智能体。

In [5]:
from autogen_agentchat.messages import HandoffMessage

# 场景：代码生成智能体无法处理复杂数学问题，移交给数学专家
math_handoff = HandoffMessage(
    content="无法解决复杂微积分问题：求函数 f(x)=x³+2x²-5x+1 的极值点",
    source="CodeGeneratorAgent",
    target="MathExpertAgent"
)

print(f"任务移交内容: {math_handoff.content}")
print(f"发送方: {math_handoff.source}")
print(f"接收方: {math_handoff.target}")

任务移交内容: 无法解决复杂微积分问题：求函数 f(x)=x³+2x²-5x+1 的极值点
发送方: CodeGeneratorAgent
接收方: MathExpertAgent


场景：软件开发流程中的阶段性移交

In [6]:
development_workflow = [
    HandoffMessage(
        content="需求分析完成，请开始系统架构设计",
        source="RequirementAnalyst",
        target="SystemArchitect"
    ),
    HandoffMessage(
        content="架构设计完成，请开始编码实现",
        source="SystemArchitect", 
        target="DeveloperAgent"
    ),
    HandoffMessage(
        content="代码开发完成，请进行测试",
        source="DeveloperAgent",
        target="TestEngineerAgent"
    )
]

for i, handoff in enumerate(development_workflow, 1):
    print(f"阶段 {i}: {handoff.source} -> {handoff.target}")
    print(f"任务: {handoff.content}\n")

阶段 1: RequirementAnalyst -> SystemArchitect
任务: 需求分析完成，请开始系统架构设计

阶段 2: SystemArchitect -> DeveloperAgent
任务: 架构设计完成，请开始编码实现

阶段 3: DeveloperAgent -> TestEngineerAgent
任务: 代码开发完成，请进行测试



场景：任务分发工厂

In [7]:
class TaskFactory:
    """任务分发工厂"""
    
    def __init__(self):
        self.specialist_mapping = {
            "code": "CodeSpecialistAgent",
            "design": "DesignSpecialistAgent", 
            "data": "DataSpecialistAgent",
            "writing": "WritingSpecialistAgent"
        }
    
    def create_handoff(self, task_type, task_description):
        if task_type in self.specialist_mapping:
            return HandoffMessage(
                content=f"Professional task assignment: {task_description}",
                source="TaskManagerAgent",
                target=self.specialist_mapping[task_type]
            )
        else:
            return HandoffMessage(
                content=f"General tasks: {task_description}",
                source="TaskManagerAgent", 
                target="GeneralAgent"
            )

# 使用示例
factory = TaskFactory()

code_task = factory.create_handoff("code", "开发一个用户登录系统")
design_task = factory.create_handoff("design", "创建移动App的UI界面")

print(f"代码任务分配给: {code_task.target}")
print(f"设计任务分配给: {design_task.target}")

代码任务分配给: CodeSpecialistAgent
设计任务分配给: DesignSpecialistAgent


**解释**：
- `content` 描述任务移交的细节。
- `source` 和 `target` 分别表示消息的发送者和接收者（目标智能体）。
- `HandoffMessage` 适用于需要动态调整任务分配的场景，例如在群聊中将任务从一个智能体移交到另一个。

2.7. `StructuredMessage` 是一个泛型类（Python 泛型（Generics）是一种允许在定义类、函数或方法时使用类型参数的编程特性，使得代码可以处理多种数据类型而保持类型安全），允许开发者定义基于 Pydantic `BaseModel` 的自定义消息类型，用于结构化的消息内容。

In [8]:
from pydantic import BaseModel # 一个用于数据验证和序列化的基类
from autogen_agentchat.messages import StructuredMessage
import datetime

# 定义一个 Pydantic 模型
class MyMessageContent(BaseModel):
    text: str
    number: datetime.datetime

# 创建一个 StructuredMessage 实例
message = StructuredMessage[MyMessageContent](
    content=MyMessageContent(text="现在的时间是：", number=datetime.datetime.now()),
    source="User",
    format_string="Hello~ {text} {number}!"
)

# 转换为文本表示
print(message.to_text())

Hello~ 现在的时间是： 2025-05-28 16:35:11.767463!


前面说到，在当前版本的 AutoGen [(0.5.7 stable)](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/index.html)版本中， `ToolCallSummaryMessage` 的 `content` 只能接收接收字符串类型，我们也可以通过继承 `StructuredMessage` 来实现更复杂的消息内容。

In [9]:
from pydantic import BaseModel
from autogen_agentchat.messages import StructuredMessage

# 定义工具调用结果的结构
class ToolCallResult(BaseModel):
    tool: str
    result: int

# 创建结构化消息
tool_call_summary = StructuredMessage[ToolCallResult](
    content=ToolCallResult(tool="calculate_sum", result=42),
    source="CalculatorAgent"
)

# 打印消息内容和来源
print(f"Content: {tool_call_summary.content}")
print(f"Source: {tool_call_summary.source}")
print(f"Text representation: {tool_call_summary.to_text()}")

Content: tool='calculate_sum' result=42
Source: CalculatorAgent
Text representation: {"tool":"calculate_sum","result":42}


2.7. **总结与补充说明**

- **ChatMessage 联合类型**：`ChatMessage` 是所有内置 `BaseChatMessage` 子类的联合类型，定义为：
  ```python
  ChatMessage = Annotated[
      TextMessage | MultiModalMessage | StopMessage | ToolCallSummaryMessage | HandoffMessage,
      Field(discriminator="type")
  ]
  ```
  这意味着任何 `BaseChatMessage` 的实例都属于上述子类之一（不包括 `StructuredMessage`）。

- **自定义消息类型**：开发者可以通过继承 `BaseChatMessage` 或使用 `StructuredMessage` 创建自定义消息类型。例如，创建一个新的消息类型需要定义必要的字段（如 `content` 和 `source`）并实现序列化逻辑。

- **消息处理**：智能体通过 `on_messages()` 或 `on_messages_stream()` 方法处理这些消息（后续会讲到）。`BaseChatMessage` 的子类通过 `type` 字段区分，方便运行时根据消息类型分发处理逻辑。

- **注意事项**：
  - 消息是纯粹的数据对象，不应包含逻辑，逻辑由智能体的消息处理方法实现。
  - 如果需要扩展消息功能，建议使用 `StructuredMessage` 结合 Pydantic 模型，以保持灵活性和类型安全性。

### 3. Agent's Internal Events 智能体内部事件

智能体内部事件继承自基类 `BaseAgentEvent`，用于记录智能体运行过程中的重要状态和操作信息。这些事件对于调试、监控和分析智能体行为非常有用。

主要的内部事件类型包括：
- `ToolCallRequestEvent` - 记录工具调用  
- `ToolCallExecutionEvent` - 记录内容过滤

3.2. `ToolCallEvent` 记录智能体调用工具函数的相关信息，包括工具名称、参数、执行结果等。这对于监控工具使用情况和调试工具调用问题很有帮助。

当智能体调用工具时，AutoGen 会生成相应的 `ToolCallEvent`。

In [21]:
import os
import sys
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import ToolCallRequestEvent, TextMessage
from autogen_core import CancellationToken 


model_client = OpenAIChatCompletionClient(
    model="gpt-4.1",  # 修正模型名称
    api_key=os.getenv("OPENAI_API_KEY"),
)

# 定义一个工具函数，让智能体调用
async def hello_world(name: str) -> str:
    """
    A simple tool that returns a greeting message.
    :param name: The name of the person to greet.
    """
    return f"Hello {name}~ Welcome to WTF-Agent!"

# 创建智能体
agent = AssistantAgent(
    name="Receptionist",
    model_client=model_client,
    tools=[hello_world],
    system_message="You are a warm receptionist. Please use the tools to help your friends.",
)



async def handle_tool_call_events():
    """处理工具调用事件的函数"""
    print("\n=== 开始智能体交互，监听工具调用事件 ===")
    
    # 创建任务消息
    task = "和 'WTF-Agent 爱好者' 打个招呼。"
    print(f"用户请求: {task}")

    # 创建取消令牌
    cancellation_token = CancellationToken()

    # 创建正确的消息格式 - 使用 TextMessage 对象而不是字典
    user_message = TextMessage(content=task, source="User")
    
    # 使用流式接口运行智能体并捕获事件
    async for message in agent.on_messages_stream([user_message],
                                                    cancellation_token=cancellation_token):
        # 检查是否是 ToolCallRequestEvent
        if isinstance(message, ToolCallRequestEvent):
            print(f"\n[事件监听器] 接收到工具调用请求事件:")
            print(f"  事件类型: {type(message).__name__}")
            print(f"  源: {message.source}")
            print(f"  工具调用ID: {getattr(message, 'call_id', 'N/A')}")
            print(f"  工具名称: {getattr(message, 'tool_name', 'N/A')}")
            print(f"  调用参数: {getattr(message, 'arguments', 'N/A')}")
        else:
            # 处理其他消息类型
            print(f"\n[消息] {type(message).__name__}: {getattr(message, 'content', str(message))}")

# 运行事件处理
await handle_tool_call_events()



=== 开始智能体交互，监听工具调用事件 ===
用户请求: 和 'WTF-Agent 爱好者' 打个招呼。

[事件监听器] 接收到工具调用请求事件:
  事件类型: ToolCallRequestEvent
  源: Receptionist
  工具调用ID: N/A
  工具名称: N/A
  调用参数: N/A

[消息] ToolCallExecutionEvent: [FunctionExecutionResult(content='Hello WTF-Agent 爱好者~ Welcome to WTF-Agent!', name='hello_world', call_id='call_p1OB5rUT2WxJwFqvkSmhUAyV', is_error=False)]

[消息] Response: Response(chat_message=ToolCallSummaryMessage(source='Receptionist', models_usage=None, metadata={}, content='Hello WTF-Agent 爱好者~ Welcome to WTF-Agent!', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='Receptionist', models_usage=RequestUsage(prompt_tokens=95, completion_tokens=20), metadata={}, content=[FunctionCall(id='call_p1OB5rUT2WxJwFqvkSmhUAyV', arguments='{"name":"WTF-Agent 爱好者"}', name='hello_world')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Receptionist', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Hello WTF-Agent 爱好者~ Welcome to WTF-Agen

**解释**：
- `ToolCallEvent` 在智能体调用任何工具函数时自动生成
- 记录工具名称、输入参数、执行结果和执行状态
- 包含详细的调用时间戳，便于性能分析
- 帮助开发者追踪工具使用模式和排查工具相关问题

3.5. **真实场景中的事件监听实现**

在生产环境中，可以这样实现事件监听和处理：